In [1]:
import reset
from pathlib import Path
import numpy as np
import scipy as sp
import tifffile
import math
import matplotlib.pyplot as plt

In [2]:
# Assign variables from reset
animal_loc = reset.animal_loc
animal_id = reset.animal_id
hemi = reset.hemi
divisor = reset.divisor
down_x = reset.down_x
down_y = reset.down_y
loc_max = reset.loc_max
grid = reset.grid
frame_len = reset.frame_len

In [3]:
# Create list of locations as strings
loc_list = [str(n+1) for n in range(loc_max)]

# Define paths for input and output images
image_loc = animal_loc / "trimmed" / hemi
save_loc = animal_loc / "xmerge" / hemi

# Ensure save location exists
save_loc.mkdir(parents=True, exist_ok=True)

image_path_list = [f"{image_loc}/{animal_id}_{hemi}_{loc}.tif" for loc in loc_list]

In [4]:

# Define function for aligning x positions based on maximum correlation
def x_connector(fn, path_num1, path_num2, max_list, start_x_list, rect_x, rect_x_move, rect_x_start):
    # Load and extract frames from both images
    im1 = tifffile.imread(image_path_list[path_num1 - 1])
    im1_f = im1[fn, :, :]
    del im1

    im2 = tifffile.imread(image_path_list[path_num2 - 1])
    im2_f = im2[fn, :, :]
    del im2

    # Calculate correlations over specified range
    cor = [
        np.corrcoef(
            im1_f[:, :rect_x].ravel(),
            im2_f[:, i:i + rect_x].ravel()
        )[0, 1]
        for i in range(rect_x_start, rect_x_start + rect_x_move)
    ]

    # Determine start position with max correlation
    start_x = np.argmax(cor) + rect_x
    max_list.append(max(cor))
    start_x_list.append(start_x)

def three_sections(f1, f2, f3, frame_name, rect_x, rect_x_move, rect_x_start):
    global max1_list, max2_list, start_x1_list, start_x2_list

    max1_list, max2_list = [], []
    start_x1_list, start_x2_list = [], []

    # Loop over frames and call x_connector for all frames
    for fn in range(frame_len):
        x_connector(fn, f1, f2, max1_list, start_x1_list, rect_x, rect_x_move, rect_x_start)
        x_connector(fn, f2, f3, max2_list, start_x2_list, rect_x, rect_x_move, rect_x_start)

    # Find target frames and starting x positions for maximum correlations
    target1_fn = np.argmax(max1_list)
    target1_start_x = start_x1_list[target1_fn]
    target2_fn = np.argmax(max2_list)
    target2_start_x = start_x2_list[target2_fn]
    print("First section target frame and start x:", target1_fn, target1_start_x)
    print("Second section target frame and start x:", target2_fn, target2_start_x)

    # Load and align images in frame_name for all frames
    for fn in range(frame_len):
        im1 = tifffile.imread(image_path_list[f1 - 1])
        rot_im1 = im1[fn, ::-1, ::-1]
        frame_name[fn, :, :width] = rot_im1
        del im1

        im2 = tifffile.imread(image_path_list[f2 - 1])
        rot_im2 = im2[fn, ::-1, ::-1]
        frame_name[fn, :, width:2 * width - target1_start_x] = rot_im2[:, target1_start_x:]
        del im2

        im3 = tifffile.imread(image_path_list[f3 - 1])
        rot_im3 = im3[fn, ::-1, ::-1]
        frame_name[fn, :, 2 * width - target1_start_x:3 * width - target1_start_x - target2_start_x] = rot_im3[:, target2_start_x:]
        del im3

In [5]:
rect_x = 35
rect_x_move = 100
rect_x_start = down_x - rect_x - rect_x_move

width = down_x
frame_height = down_y
frame_width = 3 * width

# Initialize frames for top, middle, and bottom sections
frame_top = np.zeros((frame_len, frame_height, frame_width))
frame_middle = np.zeros((frame_len, frame_height, frame_width))
frame_bottom = np.zeros((frame_len, frame_height, frame_width))

In [6]:
# Run alignment on specified sections
three_sections(7, 8, 9, frame_top, rect_x, rect_x_move, rect_x_start)
# Save the aligned image
save_path = save_loc / f"{animal_id}_{hemi}_789.tif"
tifffile.imwrite(save_path, frame_top)
print(f"Aligned image saved to {save_path}")

First section target frame and start x: 26 130
Second section target frame and start x: 19 87
Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerge/l/rbak006_l_789.tif


In [7]:
three_sections(1, 2, 3, frame_top, rect_x, rect_x_move, rect_x_start)
save_path = save_loc / f"{animal_id}_{hemi}_123.tif"
tifffile.imwrite(save_path, frame_top)
print(f"Aligned image saved to {save_path}")

First section target frame and start x: 60 126
Second section target frame and start x: 6 39
Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerge/l/rbak006_l_123.tif


In [8]:
three_sections(6, 5, 4, frame_top, rect_x, rect_x_move, rect_x_start)
save_path = save_loc / f"{animal_id}_{hemi}_654.tif"
tifffile.imwrite(save_path, frame_top)
print(f"Aligned image saved to {save_path}")

First section target frame and start x: 26 87
Second section target frame and start x: 60 129
Aligned image saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/xmerge/l/rbak006_l_654.tif


In [ ]:
# Display the aligned image
plt.imshow(frame_top[0,:,:])
plt.show()